In [1]:
!pip install elasticsearch

In [2]:
from datetime import datetime
from elasticsearch import Elasticsearch

ELASTIC_PASSWORD = "p2iFCHUbC7ze1QoIMVw"

es = Elasticsearch("http://elasticsearch:9200",
                    basic_auth=("elastic", ELASTIC_PASSWORD))

es.info()

ObjectApiResponse({'name': 'es-node', 'cluster_name': 'tdt4117-ir-data-cluster', 'cluster_uuid': 'LUzQcSqTSK-pOXCOen4SGA', 'version': {'number': '8.4.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '89f8c6d8429db93b816403ee75e5c270b43a940a', 'build_date': '2022-09-14T16:26:04.382547801Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [3]:
!mkdir enron && cd enron && wget https://www.cs.cmu.edu/~enron/enron_mail_20150507.tar.gz && tar xf enron_mail_20150507.tar.gz

mkdir: cannot create directory ‘enron’: File exists


In [4]:
from pathlib import Path
import shutil

files = Path("enron/maildir").glob("**/*")

# limit number of email to 100000

count = 0
for i, f in enumerate(files):
    if count >= 100000:
        break
    if f.is_file():
        with open(f, "r") as text:
            try:
                text.read()
                fn = str(f).replace('enron', 'enron_short')
                Path(fn).mkdir(parents=True, exist_ok=True)
                shutil.move(f, fn)
                count += 1
            except:
                print(f"skipping: {i} - {f}")

skipping: 342 - enron/maildir/beck-s/deleted_items/13.
skipping: 343 - enron/maildir/beck-s/deleted_items/38.
skipping: 344 - enron/maildir/beck-s/deleted_items/299.
skipping: 345 - enron/maildir/beck-s/deleted_items/264.
skipping: 346 - enron/maildir/beck-s/deleted_items/91.
skipping: 347 - enron/maildir/beck-s/deleted_items/353.
skipping: 348 - enron/maildir/beck-s/deleted_items/48.
skipping: 349 - enron/maildir/beck-s/deleted_items/85.
skipping: 350 - enron/maildir/beck-s/deleted_items/36.
skipping: 353 - enron/maildir/beck-s/inbox/504.
skipping: 354 - enron/maildir/beck-s/inbox/704.
skipping: 355 - enron/maildir/beck-s/inbox/100.
skipping: 357 - enron/maildir/beck-s/inbox/763.
skipping: 358 - enron/maildir/beck-s/inbox/708.
skipping: 619 - enron/maildir/shapiro-r/deleted_items/221.
skipping: 631 - enron/maildir/gay-r/all_documents/82.
skipping: 632 - enron/maildir/gay-r/all_documents/59.
skipping: 633 - enron/maildir/gay-r/sent/82.
skipping: 634 - enron/maildir/gay-r/sent/59.
skipp

In [5]:
# Index all emails folder enron_short/maildir
from pathlib import Path
from email.parser import Parser
files = Path("enron_short/maildir").glob("**/*")
i=0
for i,f in enumerate(files):
    if i >= 100000:
        break
    if f.is_file():
        if(i%500==0):
            print(i)
        with open(f,"r") as text:
            try:
                i=i+1
                txt=text.read()
                email=Parser().parsestr(txt)
                es.index(
                    index='email',
                    document={
                        "To" : email['to'],
                        "From" : email['from'],
                        "Subject" : email['subject'],
                        "Body" : email.get_payload()
                                     
                    })
                
            except:
                pass
            
es.info()

1500
3500
4000
5500
6500
10500
11000
11500
12000
12500
13000
14000
15500
16500
19500
20000
20500
21000
21500
22000
24000
24500
25000
26500
27000
28000
29500
30000
31000
32000
32500
33500
34000
34500
38500
39000
39500
40000
40500
41000
41500
42000
42500
45500
46000
46500
49000
49500
50000
50500
51500
53500
54000
54500
55000
57500
58000
58500
59000
60000
62000
62500
63000
64500
65000
65500
66500
67000
69500
70000
70500
71000
71500
72500
74000
74500
76000
76500
77000
77500
78000
79000
81000
81500
85000
85500
86000
86500
87000
89000
89500
90000
91500
92000
92500
94000
95000
95500
96500
97500
98500


ObjectApiResponse({'name': 'es-node', 'cluster_name': 'tdt4117-ir-data-cluster', 'cluster_uuid': 'LUzQcSqTSK-pOXCOen4SGA', 'version': {'number': '8.4.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '89f8c6d8429db93b816403ee75e5c270b43a940a', 'build_date': '2022-09-14T16:26:04.382547801Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [ ]:
# Query for "Norwegian and University and Science and Technology” and "Norwegian University Science Technology"

In [7]:
es.indices.refresh(index='email')
result = es.search(
 index='email',
  query={
    'match': {'Body': 'I'}
  }
 )

result['hits']['hits']

[{'_index': 'email',
  '_id': 'gblrM4QB0mxI_Z1IXv-b',
  '_score': 1.291624,
  '_ignored': ['Body.keyword', 'To.keyword'],
  '_source': {'To': 'christopher.bruner@motorola.com, bruners@uswest.net, maurden@rconnect.com, \n\tdmcnemar@juno.com, lzzylou8@juno.com, gwlee@mymailstation.com, \n\tsjbruner@worldnet.att.net, misterpiccalo@hotmail.com, \n\tmwoita@home.com, wpatti0205@aol.com, mark.woita@fhlbtopeka.com, \n\tgjforman@juno.com, corey-ponder@ouhsc.edu, helenswig@aol.com, \n\tjjjsmd@aol.com, jim.schwieger@enron.com, smithki@ms.com',
   'From': 'mark.woita@fhlbtopeka.com',
   'Subject': "FW: When you thought I wasn't looking...",
   'Body': 'Family,\nThought you might enjoy this.\n-Mark\n\n\n> When you thought I wasn\'t looking (by a child)...\n>\n>\n>\n> When you thought I wasn\'t looking, I saw you hang my first painting on the\n> refrigerator, and I immediately wanted to paint another one.\n>\n> When you thought I wasn\'t looking I saw you feed a stray cat, and I\n> learned that it w

In [13]:
result = es.search(
 index='email',
  query={
    'match': {'Body': 'Norwegian University Science Technology'}
  }
 )

result['hits']['hits']

[{'_index': 'email',
  '_id': 'M7lqM4QB0mxI_Z1IP9wx',
  '_score': 19.782507,
  '_ignored': ['Body.keyword'],
  '_source': {'To': 'all.houston@enron.com, gas.operations@enron.com',
   'From': 'enron.announcements@enron.com',
   'Subject': 'GSS Organizational Changes',
   'Body': 'I am pleased to announce the following organizational changes within the=20\nEnron Global Strategic Sourcing Operations Department:\n\nPeter Goebel and Roy Hartstein have joined the group as sourcing portfolio=\n=20\nleaders.  Prior to joining Enron, Peter served as commodity manager for ARC=\nO=20\nPipe Line Company.  He holds a Bachelor of Arts degree in finance and a J.D=\n.=20\ndegree from St. Mary=01,s University in San Antonio.  Roy joins Enron with =\n18=20\nyears of experience in the energy business and most recently served as=20\ndirector of materials management for Duke Energy Corporation.  He holds a=\n=20\nBachelor of Science degree in civil engineering and a M.B.A. in finance fro=\nm=20\nLouisiana 

In [28]:
result = es.search(
 index='email',
  query= {
    "match" : {
      "Body" : "Norwegian University Science Technology",

    }
  }
    
)
result

ObjectApiResponse({'took': 8, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 9934, 'relation': 'eq'}, 'max_score': 19.782507, 'hits': [{'_index': 'email', '_id': 'M7lqM4QB0mxI_Z1IP9wx', '_score': 19.782507, '_ignored': ['Body.keyword'], '_source': {'To': 'all.houston@enron.com, gas.operations@enron.com', 'From': 'enron.announcements@enron.com', 'Subject': 'GSS Organizational Changes', 'Body': 'I am pleased to announce the following organizational changes within the=20\nEnron Global Strategic Sourcing Operations Department:\n\nPeter Goebel and Roy Hartstein have joined the group as sourcing portfolio=\n=20\nleaders.  Prior to joining Enron, Peter served as commodity manager for ARC=\nO=20\nPipe Line Company.  He holds a Bachelor of Arts degree in finance and a J.D=\n.=20\ndegree from St. Mary=01,s University in San Antonio.  Roy joins Enron with =\n18=20\nyears of experience in the energy business and most recently se